In [1]:
import MetaTrader5 as mt5
from datetime import datetime
import pandas as pd
import time 
import numpy as np
from scipy.stats import norm
from scipy.optimize import brentq
import warnings

warnings.filterwarnings('ignore')

In [2]:

def black_scholes_price(S, K, T, r, sigma, option_type):
    """Calcula preço da opção pelo modelo Black-Scholes"""
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    if option_type == 'Call':
        return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    else:  # Put
        return K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)

def implied_volatility(market_price, S, K, T, r, option_type):
    """Calcula volatilidade implícita usando preço de mercado (Last, Bid, Ask)"""
    if T <= 0 or S <= 0 or K <= 0 or market_price <= 0:
        return np.nan
    try:
        func = lambda sigma: black_scholes_price(S, K, T, r, sigma, option_type) - market_price
        return brentq(func, 1e-6, 3.0)  # tenta encontrar sigma entre 0.000001 e 3.0
    except (ValueError, RuntimeError):
        return np.nan

def calcular_gregas_blackscholes_com_iv(df,vol = 0.14, taxa_r=0.10):
    df = df.copy()
    df['Data'] = pd.to_datetime(df['Data'], errors='coerce')
    df['Vencimento'] = pd.to_datetime(df['Vencimento'], errors='coerce')

    df = df.dropna(subset=['Data', 'Vencimento', 'Cotacao_Ativo', 'Strike'])
    df['Dias_para_Venc'] = (df['Vencimento'] - df['Data']).dt.days
    df['T'] = df['Dias_para_Venc'] / 252

    df = df[(df['T'] > 0) & (df['Cotacao_Ativo'] > 0) & (df['Strike'] > 0)]

    S = df['Cotacao_Ativo'].astype(float)
    K = df['Strike'].astype(float)
    T = df['T'].astype(float)
    r = taxa_r
    tipo = df['Tipo']

    # Calcula volatilidade implícita com base em Last, Bid, Ask
    df['IV_Last'] = [
        implied_volatility(p, s, k, t, r, o)
        for p, s, k, t, o in zip(df['Last'], S, K, T, tipo)
    ]
    df['IV_Bid'] = [
        implied_volatility(p, s, k, t, r, o)
        for p, s, k, t, o in zip(df['Bid'], S, K, T, tipo)
    ]
    df['IV_Ask'] = [
        implied_volatility(p, s, k, t, r, o)
        for p, s, k, t, o in zip(df['Ask'], S, K, T, tipo)
    ]

    # Use IV_Last como referência para gregas e preço teórico
    sigma_est = vol
    d1_est = (np.log(S / K) + (r + 0.5 * sigma_est ** 2) * T) / (sigma_est * np.sqrt(T))
    d2_est = d1_est - sigma_est * np.sqrt(T)

    # Use IV_Last como referência para gregas e preço teórico
    sigma = df['IV_Last']
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    # Valor justo
    df['BS_Preco_IV'] = np.where(
        df['Tipo'] == 'Call',
        S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2),
        K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    )
    # Valor justo
    df['BS_Preco_Est'] = np.where(
        df['Tipo'] == 'Call',
        S * norm.cdf(d1_est) - K * np.exp(-r * T) * norm.cdf(d2_est),
        K * np.exp(-r * T) * norm.cdf(-d2_est) - S * norm.cdf(-d1_est)
    )
    # Gregas
    df['Delta'] = np.where(
        df['Tipo'] == 'Call',
        norm.cdf(d1),
        -norm.cdf(-d1)
    )
    df['Gamma'] = norm.pdf(d1) / (S * sigma * np.sqrt(T))
    df['Vega'] = S * norm.pdf(d1) * np.sqrt(T) 
    df['Theta'] = np.where(
        df['Tipo'] == 'Call',
        (-S * norm.pdf(d1) * sigma / (2 * np.sqrt(T)) - r * K * np.exp(-r * T) * norm.cdf(d2)) / 252,
        (-S * norm.pdf(d1) * sigma / (2 * np.sqrt(T)) + r * K * np.exp(-r * T) * norm.cdf(-d2)) / 252
    )
    df['Rho'] = np.where(
        df['Tipo'] == 'Call',
        K * T * np.exp(-r * T) * norm.cdf(d2) / 100,
        -K * T * np.exp(-r * T) * norm.cdf(-d2) / 100
    )

    return df


In [3]:
# String inicial
base_string = 'BOVA'

# Gerar a lista com as variações de 'VALEA' até 'VALEZ'
result = [base_string + chr(i) for i in range(ord('A'), ord('Z') + 1)]

# Exibir o resultado
print(result[23])

BOVAX


In [4]:
mt5.initialize()

mt5.login(12712431,'Hammstein08*', 'Rico-PRD')
lista_opcoes = []


for nome in result:
    ticker = mt5.symbols_get(nome)
    # print(len(ticker))
    for j in range(1,len(ticker)):
        lista_opcoes.append(ticker[j].name)

In [5]:
for i in range(len(lista_opcoes)):
    ticker = lista_opcoes[i]
    
    # Obter as informações do símbolo de uma vez
    mt5.symbol_select(ticker,True)

time.sleep(10)
    

In [6]:
dicionario_opcoes = {}
for i in range(len(lista_opcoes)):
    ticker = lista_opcoes[i]
    
    # Obter as informações do símbolo de uma vez
    
    symbol_info = mt5.symbol_info(ticker)
    ativo_base = mt5.symbol_info(ticker).basis
    mt5.symbol_select(ativo_base,True)
    if (ativo_base == ''):
        cota = 0
    else:
        cota = mt5.symbol_info(ativo_base).last


    if symbol_info is not None:
        # Construir o valor do dicionário com as informações
        dicionario_opcoes[ticker] = {
            'Last': symbol_info.last,
            'Bid': symbol_info.bid,
            'Ask': symbol_info.ask,
            'Volume': symbol_info.session_volume,
            'Data': datetime.fromtimestamp(symbol_info.time).strftime('%Y-%m-%d'),
            'Vencimento': datetime.fromtimestamp(symbol_info.expiration_time).strftime('%Y-%m-%d'),
            'Strike': symbol_info.option_strike,
            'Cotacao_Ativo' : cota,
            'Tipo': symbol_info.option_right,
            # 'Vencimento': symbol_info.expiration_time,
            'Abertura': symbol_info.session_open
        }

# Exibir o dicionário para conferir
print(dicionario_opcoes)

grade_opcoes = pd.DataFrame(dicionario_opcoes).T

opcoes_negociadas = grade_opcoes[grade_opcoes['Volume'] != 0]
opcoes_negociadas.loc[opcoes_negociadas['Tipo'] == 0, 'Tipo'] = 'Call'
opcoes_negociadas.loc[opcoes_negociadas['Tipo'] == 1, 'Tipo'] = 'Put'

df_opcoes = opcoes_negociadas.sort_values(by='Vencimento')

{'BOVAA100': {'Last': 0.0, 'Bid': 0.0, 'Ask': 0.0, 'Volume': 0.0, 'Data': '1969-12-31', 'Vencimento': '2026-01-16', 'Strike': 10.0, 'Cotacao_Ativo': 134.63, 'Tipo': 0, 'Abertura': 0.0}, 'BOVAA119': {'Last': 0.0, 'Bid': 0.0, 'Ask': 0.0, 'Volume': 0.0, 'Data': '2025-07-09', 'Vencimento': '2027-01-15', 'Strike': 119.0, 'Cotacao_Ativo': 134.63, 'Tipo': 0, 'Abertura': 0.0}, 'BOVAA125': {'Last': 21.87, 'Bid': 10.0, 'Ask': 0.0, 'Volume': 0.0, 'Data': '2025-07-09', 'Vencimento': '2026-01-16', 'Strike': 125.0, 'Cotacao_Ativo': 134.63, 'Tipo': 0, 'Abertura': 0.0}, 'BOVAA126': {'Last': 0.0, 'Bid': 0.01, 'Ask': 0.0, 'Volume': 0.0, 'Data': '2025-07-09', 'Vencimento': '2026-01-16', 'Strike': 126.0, 'Cotacao_Ativo': 134.63, 'Tipo': 0, 'Abertura': 0.0}, 'BOVAA127': {'Last': 0.0, 'Bid': 0.0, 'Ask': 0.0, 'Volume': 0.0, 'Data': '1969-12-31', 'Vencimento': '2026-01-16', 'Strike': 127.0, 'Cotacao_Ativo': 134.63, 'Tipo': 0, 'Abertura': 0.0}, 'BOVAA128': {'Last': 0.0, 'Bid': 0.0, 'Ask': 0.0, 'Volume': 0.0, '

In [7]:
df_opcoes

,Last,Bid,Ask,Volume,Data,Vencimento,Strike,Cotacao_Ativo,Tipo,Abertura
BOVAS146W2,10.8,11.18,11.24,69454.0,2025-07-09,2025-07-11,146.0,134.63,Put,10.86
BOVAG132W2,2.83,2.82,2.89,169434.0,2025-07-09,2025-07-11,132.0,134.63,Call,4.03
BOVAS132W2,0.1,0.08,0.08,1024998.0,2025-07-09,2025-07-11,132.0,134.63,Put,0.08
BOVAG131W2,3.75,3.79,3.85,50058.0,2025-07-09,2025-07-11,131.0,134.63,Call,5.1
BOVAS131W2,0.06,0.04,0.06,141220.0,2025-07-09,2025-07-11,131.0,134.63,Put,0.04
...,...,...,...,...,...,...,...,...,...,...
BOVAX145,7.38,0.0,0.0,256.0,2025-07-08,2026-12-18,145.0,134.63,Put,7.4
BOVAX13,4.55,3.47,4.5,650.0,2025-07-09,2026-12-18,130.0,134.63,Put,4.5
BOVAX139,6.51,6.21,7.2,301.0,2025-07-09,2026-12-18,140.0,134.63,Put,6.51
BOVAP175,16.8,16.25,18.0,9198.0,2025-07-09,2027-04-16,175.0,134.63,Put,16.2


In [8]:
calcular_gregas_blackscholes_com_iv(df_opcoes)

,Last,Bid,Ask,Volume,Data,Vencimento,Strike,Cotacao_Ativo,Tipo,Abertura,...,IV_Last,IV_Bid,IV_Ask,BS_Preco_IV,BS_Preco_Est,Delta,Gamma,Vega,Theta,Rho
BOVAS146W2,10.8,11.18,11.24,69454.0,2025-07-09,2025-07-11,146.0,134.63,Put,10.86,...,NaN,NaN,NaN,NaN,11.254173,NaN,NaN,NaN,NaN,NaN
BOVAG132W2,2.83,2.82,2.89,169434.0,2025-07-09,2025-07-11,132.0,134.63,Call,4.03,...,0.176958,0.171907,0.203391,2.83,2.769397,0.904844,0.079732,2.029625,-0.137007,0.009444
BOVAS132W2,0.1,0.08,0.08,1024998.0,2025-07-09,2025-07-11,132.0,134.63,Put,0.08,...,0.179258,0.169130,0.169130,0.10,0.034676,-0.097997,0.080428,2.073958,-0.087668,-0.001055
BOVAG131W2,3.75,3.79,3.85,50058.0,2025-07-09,2025-07-11,131.0,134.63,Call,5.1,...,0.158206,0.198713,0.235420,3.75,3.740835,0.977394,0.028302,0.644104,-0.076204,0.010146
BOVAS131W2,0.06,0.04,0.06,141220.0,2025-07-09,2025-07-11,131.0,134.63,Put,0.04,...,0.201601,0.185569,0.201601,0.06,0.006908,-0.057623,0.047727,1.384120,-0.066658,-0.000620
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BOVAX145,7.38,0.0,0.0,256.0,2025-07-08,2026-12-18,145.0,134.63,Put,7.4,...,0.197544,NaN,NaN,7.38,3.835226,-0.268877,0.008571,64.300415,0.005265,-0.913082
BOVAX13,4.55,3.47,4.5,650.0,2025-07-09,2026-12-18,130.0,134.63,Put,4.5,...,0.215223,0.192841,0.214221,4.55,1.332824,-0.173615,0.006121,49.933847,0.000885,-0.583961
BOVAX139,6.51,6.21,7.2,301.0,2025-07-09,2026-12-18,140.0,134.63,Put,6.51,...,0.206557,0.201538,0.217967,6.51,2.795656,-0.236217,0.007663,59.999703,0.003445,-0.801206
BOVAP175,16.8,16.25,18.0,9198.0,2025-07-09,2027-04-16,175.0,134.63,Put,16.2,...,0.190867,0.184416,0.204959,16.80,12.452519,-0.446911,0.009611,85.231395,0.017951,-1.973059


In [15]:
df_comp = calcular_gregas_blackscholes_com_iv(df_opcoes, vol = 0.13480,taxa_r=0.1490)
df_comp['UpSide'] = df_comp['Last'] - df_comp['BS_Preco_Est']
df_comp = df_comp[['Last', 'Bid', 'Ask', 'Volume', 'Vencimento', 'Strike','Cotacao_Ativo', 'Tipo', 'Dias_para_Venc', 'IV_Last', 'IV_Bid', 'IV_Ask', 'BS_Preco_IV', 'BS_Preco_Est','UpSide', 'Delta', 'Gamma', 'Vega', 'Theta', 'Rho']]



df_comp.loc[(df_comp['Dias_para_Venc'] >=15) & (df_comp['Dias_para_Venc'] <= 45) ]

,Last,Bid,Ask,Volume,Vencimento,Strike,Cotacao_Ativo,Tipo,Dias_para_Venc,IV_Last,IV_Bid,IV_Ask,BS_Preco_IV,BS_Preco_Est,UpSide,Delta,Gamma,Vega,Theta,Rho
BOVAS138W4,3.33,3.24,3.3,3660.0,2025-07-25,138.0,134.63,Put,16,0.156711,0.149573,0.154337,3.33,3.056090,0.27391,-0.643111,0.070161,12.653139,-0.008803,-0.057087
BOVAG138W4,0.76,0.73,0.77,23036.0,2025-07-25,138.0,134.63,Call,16,0.116212,0.113684,0.117051,0.76,0.985458,-0.225458,0.306221,0.089008,11.903757,-0.067157,0.025693
BOVAG130W4,5.66,5.68,5.76,15471.0,2025-07-25,130.0,134.63,Call,16,NaN,NaN,NaN,NaN,6.053849,-0.393849,NaN,NaN,NaN,NaN,NaN
BOVAS122W4,0.06,0.06,0.09,120.0,2025-07-25,122.0,134.63,Put,16,0.215131,0.215131,0.230052,0.06,0.000868,0.059132,-0.021751,0.007123,1.763427,-0.010088,-0.001897
BOVAG144W4,0.1,0.08,0.1,48482.0,2025-07-25,144.0,134.63,Call,16,0.139018,0.133061,0.139018,0.10,0.085580,0.01442,0.051215,0.022292,3.566301,-0.019511,0.004314
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BOVAH140W4,2.11,1.0,0.0,274.0,2025-08-22,140.0,134.63,Call,44,0.128907,0.077281,NaN,2.11,2.239368,-0.129368,0.414415,0.053742,21.924406,-0.063857,0.093732
BOVAH142W4,0.88,0.0,2.59,30.0,2025-08-22,142.0,134.63,Call,44,0.099906,NaN,0.184259,0.88,1.557733,-0.677733,0.263490,0.058110,18.372712,-0.041313,0.060402
BOVAH130W4,6.0,0.0,9.6,2.0,2025-08-22,130.0,134.63,Call,44,NaN,NaN,0.208952,NaN,8.490063,-2.490063,NaN,NaN,NaN,NaN,NaN
BOVAT124W4,0.25,0.0,0.0,2.0,2025-08-22,124.0,134.63,Put,45,0.168654,NaN,NaN,0.25,0.077999,0.172001,-0.059024,0.012257,6.690554,-0.007691,-0.014636


In [12]:
df_comp.columns

Index(['Last', 'Bid', 'Ask', 'Volume', 'Data', 'Vencimento', 'Strike',
       'Cotacao_Ativo', 'Tipo', 'Abertura', 'Dias_para_Venc', 'T', 'IV_Last',
       'IV_Bid', 'IV_Ask', 'BS_Preco_IV', 'BS_Preco_Est', 'Delta', 'Gamma',
       'Vega', 'Theta', 'Rho', 'UpSide'],
      dtype='object')

In [ ]:
mt5.symbol_select(ativo_base)

dados = mt5.copy_rates_from_pos(ativo_base, mt5.TIMEFRAME_D1, 0, 180)

# 4. Converte para DataFrame e ajusta datas
df = pd.DataFrame(dados)
df['time'] = pd.to_datetime(df['time'], unit='s')
print(df[['time', 'open', 'high', 'low', 'close', 'tick_volume']])

          time    open    high     low   close  tick_volume
0   2024-10-15  127.50  127.80  126.75  127.40        74336
1   2024-10-16  127.73  128.65  127.27  128.07        81708
2   2024-10-17  126.60  127.43  126.40  127.31        69809
3   2024-10-18  128.12  128.23  126.66  127.00       111994
4   2024-10-21  127.36  127.62  126.66  126.75       134049
..         ...     ...     ...     ...     ...          ...
175 2025-07-03  136.45  138.15  136.09  137.92        77130
176 2025-07-04  137.39  138.45  137.39  138.13        29900
177 2025-07-07  138.05  138.09  136.17  136.45        72051
178 2025-07-08  136.34  136.40  135.66  136.15        39229
179 2025-07-09  135.94  135.94  134.38  134.65        39479

[180 rows x 6 columns]


In [ ]:
(df['close'].pct_change().rolling(21).std())*np.sqrt(252)

0           NaN
1           NaN
2           NaN
3           NaN
4           NaN
         ...   
175    0.126200
176    0.124515
177    0.131982
178    0.132200
179    0.138302
Name: close, Length: 180, dtype: float64

In [ ]:
# LIMPA A MARKWT WATCH DO MT5

lista_remocao = list(grade_opcoes.index)

for nome in lista_remocao:
    mt5.symbol_select(nome,False)

mt5.symbol_select(ativo_base,False)

    

True

In [ ]:

mt5.shutdown()


True